In [22]:
import tensorflow as tf
import numpy as np
import pandas as pd
import math as mt
import os
import pickle

In [23]:
import copy

In [24]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [25]:
df=pd.read_csv("data/labl.csv")
df.head()


,image,class,is_valid
0,ISIC_0024310_0,0,False
1,ISIC_0024310_1,0,False
2,ISIC_0024310_2,0,False
3,ISIC_0024310_3,0,False
4,ISIC_0024310_4,0,False


In [26]:
vals=df.groupby(["is_valid","class"]).count()

In [27]:
print(vals)

                image
is_valid class       
False    0       6035
         1       6035
         2       6035
         3       6035
         4       6035
         5       6035
         6       6035
True     0        111
         1        670
         2         51
         3         32
         4        109
         5         11
         6         14


In [28]:
batch_size=32

In [29]:
def train_valid_test(pathcsv,trainprop,validprop,testprop,bs=batch_size):
    df=pd.read_csv(pathcsv)
    df=df.sample(frac=1).reset_index()
    x=df["is_valid"].value_counts()
    
    tot={i:[] for i in range(7)}
    tot_test={i:[] for i in range(7)}
    x_train=[]
    x_test=[]
    x_valid=[]
    y_train=[]
    y_valid=[]
    y_test=[]
    ml=-999999
    print((x[0]//7))
    train_amnt=((int(trainprop*(x[0]//7))//bs)*bs)
    print(train_amnt)
    valid_amnt=int((int(((x[0]//7)-train_amnt)*validprop)//bs)*bs)
    # test_amnt=int((int(x[1]*testprop)//bs)*bs)
    for i,row in df.iterrows():
        if row["is_valid"]==False:
            tot[int(row["class"])].append(row["image"]+".jpg")
        else:
            tot_test[int(row["class"])].append(row["image"]+".jpg")
            # x_test.append(row["image"]+".jpg")
            # y_test.append(int(row["class"]))
        
    for v in tot_test:
        ml=max(ml,len(tot_test[v]))
        print(v,len(tot_test[v]))
    print("ml",ml)
    test_amnt=int((int(ml*testprop)//bs)*bs)
    print("test_amount",test_amnt)
    for v in tot:
        x_train+=(tot[v][:train_amnt])
        y_train+=[v]*train_amnt
        x_valid+=(tot[v][train_amnt:train_amnt+valid_amnt])
        y_valid+=[v]*valid_amnt
    for v in tot_test:
        val=copy.copy((tot_test[v][:]))
        if len(val)<=ml:
            inc=(ml//len(val))+1
            for j in range(inc):
                tot_test[v]+=val
            tot_test[v]=copy.copy(tot_test[v][:test_amnt])
        else:
            tot_test[v]=copy.copy(tot_test[v][:test_amnt])
    for v in tot_test:
        print(len(tot_test[v]))
    for v in tot_test:
        x_test+=tot_test[v]
        y_test+=[v]*test_amnt

    return x_train,y_train,x_valid,y_valid,x_test,y_test

In [30]:
pathcsv="data/labl.csv"
x_train,y_train,x_valid,y_valid,x_test,y_test=train_valid_test(pathcsv,0.16,0.05,0.15)

6035
960
0 111
1 670
2 51
3 32
4 109
5 11
6 14
ml 670
test_amount 96
96
96
96
96
96
96
96


In [31]:
print(len(x_train),len(y_train),len(x_valid),len(y_valid),len(x_test),len(y_test))

6720 6720 1568 1568 672 672


In [33]:
for i in range(7):
    cnt=0
    for v in y_test:
        if v==i:
            cnt+=1
    print(cnt)

96
96
96
96
96
96
96


In [8]:
print(x_test)

['ISIC_0028076.jpg', 'ISIC_0031738.jpg', 'ISIC_0030631.jpg', 'ISIC_0029252.jpg', 'ISIC_0030292.jpg', 'ISIC_0029036.jpg', 'ISIC_0031209.jpg', 'ISIC_0027443.jpg', 'ISIC_0030286.jpg', 'ISIC_0030936.jpg', 'ISIC_0026297.jpg', 'ISIC_0024433.jpg', 'ISIC_0031450.jpg', 'ISIC_0032977.jpg', 'ISIC_0034036.jpg', 'ISIC_0029669.jpg', 'ISIC_0026139.jpg', 'ISIC_0024355.jpg', 'ISIC_0029365.jpg', 'ISIC_0031213.jpg']


In [9]:
def find_suitable(h,w,mxval):
    gcd=mt.gcd(h,w)
    val=1
    while(val<=gcd):
        if h%val==0 and w%val==0 and (h//val)*(w/val)<=mxval:
            # print(val)
            return h//val,w//val
        val+=1

In [34]:
rh=45
rw=60

In [36]:
def preprocess_image(path,mxval,nchan=3,pemb=True):
  global rh
  global rw
  image = tf.io.read_file(path)
  image = tf.image.decode_jpeg(image, channels=nchan)
  sh=image.shape
  # if rh==None and rw==None:
  #   rh,rw=find_suitable(sh[0],sh[1],mxval)
  image = tf.image.resize(image, [rh,rw])
  # image=tf.squeeze(image)
  sh=image.shape
  image=image.numpy()
  image=np.float32(image)
  if pemb:
    numdig=len(str(sh[0]*sh[1]))
    mv=255.0*((10)**numdig)+sh[0]*sh[1]
    
    for k in range(nchan):
      cnt=1
      for i, v in enumerate(image):
         for j,val in enumerate(v):
            image[i][j][k]=image[i][j][k]*((10)**numdig)+cnt
            cnt+=1
      # print(mv,np.max(image[:][:][k]))
    image/=mv
    
    
  else:
    image=image/255
  
  # print(image.shape)
  # image=image.reshape(rw*rh,)
  # print(image.shape)
  return image

In [37]:
print(x_test[0])

ISIC_0031406.jpg


In [60]:
# for i in range(5):
#     image=preprocess_image("data/balanced_image/"+'ISIC_0033544.jpg',200,pemb=False)

In [62]:
# image.shape

(9, 12, 3)

In [38]:
# image=preprocess_image("data/balanced_image/ISIC_0033544.jpg",200)
x1=[]
x2=[]
x3=[]
y1=[]
y2=[]
y3=[]

# for i,v in enumerate(x_train):
#     try:
#         # print("data/balanced_image/"+x_train[i])
#         x1.append(preprocess_image("data/balanced_image/"+x_train[i],200))
#         y1.append(y_train[i])
#     except Exception as e:
#         print(x_train[i],y_train[i])
        
# for i,v in enumerate(x_valid):
#     try:
#         x2.append(preprocess_image("data/balanced_image/"+x_valid[i],200))
#         y2.append(y_valid[i])
#     except Exception as e:
#         print(x_valid[i],y_valid[i])
for i,v in enumerate(x_test):
    try:
        x3.append(preprocess_image("data/balanced_image/"+x_test[i],200))
        y3.append(y_test[i])
    except Exception as e:
        print(x_test[i],y_test[i])
    

In [23]:
def oh(arr,depth=7):
     return tf.one_hot(arr, depth)
     

In [40]:
x1=np.array(x1)
y1=np.float32(np.array(oh(y1)))
x2=np.array(x2)
y2=np.float32(np.array(oh(y2)))
x3=np.array(x3)
y3=np.float32(np.array(oh(y3)))

In [39]:
# x1=np.array(x1)
# y1=np.float32(np.array(y1))
# x2=np.array(x2)
# y2=np.float32(np.array(y2))
x3=np.array(x3)
y3=np.float32(np.array(y3))

In [19]:
x1.shape

(6720, 45, 60, 3)

In [10]:
# for i in range(10):
#     x1.append(preprocess_image("data/balanced_image/"+'ISIC_0033544.jpg',200))
#     y1.append(1)
#     x2.append(preprocess_image("data/balanced_image/"+'ISIC_0033544.jpg',200))
#     y2.append(1)

In [20]:
x1.shape

(6720, 45, 60, 3)

In [40]:
# with open('data/train/x_train.pkl','wb') as f:
#         pickle.dump(x1, f)
# with open('data/train/y_train.pkl','wb') as f:
#         pickle.dump(y1, f)
# with open('data/valid/x_valid.pkl','wb') as f:
#         pickle.dump(x2, f)
# with open('data/valid/y_valid.pkl','wb') as f:
#         pickle.dump(y2, f)
with open('data/test/x_test.pkl','wb') as f:
        pickle.dump(x3, f)
with open('data/test/y_test.pkl','wb') as f:
        pickle.dump(y3, f)